<a href="https://colab.research.google.com/github/AbooMardiiyah/Named-entity-recognition/blob/main/Hamzat_Tiamiyu_NLP_Week_2_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> DUPLICATE THIS COLAB TO START WORKING ON IT. Using File > Save a copy to drive.


# Week 2: Named Entity Recognition (NER)

### What we are building
We'll build a model to do NER on an open source MIT Movie Dataset. We will continue to apply our learning philosophy of repetition as we build multiple models of increasing complexity in the following order:

1. Simple model to get a baseline
1. RNN based model
1. Using LSTM. Does it do better? 
1. Using Stacked LSTMs
1. **Extension**: Explore different parameters, features, and metrics. 

###  Evaluation
We will evaluate our models along the following two dimensions: 

1. Accuracy: the ratio of the number of correctly classified instances to the total number of instances. 
1. F1: since this is a multi-class classification problem, we'll add these metrics to each of the runs using the relevant functions from torchmetrics.


### Instructions

1. We've provide scaffolding for all the boiler plate Pytorch code to get to our baseline model. This covers downloading and parsing the dataset, and training code for the baseline model. **Make sure to read all the steps and internalize what is happening**.
1. At this point, our model gets to an accuracy of about 0.78. After this we'll try to improve the model by using RNNs and LSTMs. **Does this improve accuracy?**
1. In the next set of models, we'll stack the LSTM cells and see if they can boost up our model. **How do you think this model will perform?**
1. **Extension**: We have suggested a bunch of extensions to the project so go crazy! Tweak any parts of the pipeline, and see if you can beat all the current modes.

### Code Overview
- Dependencies: Install and import python dependencies
- Project
  - Dataset: Download the MIT dataset, and parse it into a pytorch Dataset
  - Trainer: Trainer function to help with multi-epoch training
  - Model 0: All Zeros
  - Model 1: RNN
  - Model 2: Bidirectional RNN
  - Model 2: LSTM
  - Model 3: Stacked LSTMs
  - More metrics
- Extensions


# Dependencies

✨ Now let's get started! To kick things off, as always, we will install some dependencies.

In [ ]:
%%capture
# Install all the required dependencies for the project
!pip install pytorch-lightning==1.5.10
!pip install spacy==2.2.4
!python -m spacy download en_core_web_md
!pip install scikit-learn==1.0.2

Import all the necessary libraries we need throughout the project.

In [ ]:
# Import all the relevant libraries
import spacy
import en_core_web_md
import csv
import itertools

import pytorch_lightning as pl
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset, random_split
import numpy as np
from torch.nn.utils.rnn import pad_sequence
from sklearn.preprocessing import LabelEncoder
import torchmetrics

Now let's load the Spacy data, which comes with pre-trained embeddings. This process is expensive so only do this once.

In [ ]:
# Really expensive operation to load the entire space word-vector index in memory
# We'll only run it once 
loaded_spacy_model = en_core_web_md.load()

Fix the random seed for numpy and pytorch so the entire class gets consistent results which. We can discuss this with each other.

In [ ]:
# Fix the random seed so that we get consistent results
torch.manual_seed(0)
np.random.seed(0)

# Named Entity Recognition Project 

✨ Let's Begin ✨

### Data Loading and Processing (Common to ALL Solutions)

#### Dataset

We’ll be using the [MIT Movie dataset](https://groups.csail.mit.edu/sls/downloads/movie/). **Extension**: You can find a lot of other entity recognition datasets listed on Github. Play around with them to see how you perform.


In [ ]:
# Download the dataset and move it to a separate folder
!mkdir -p mit_ner
# !mkdir -p pcm_ner
!wget 'https://groups.csail.mit.edu/sls/downloads/movie/engtrain.bio' -P './mit_ner'
!wget 'https://groups.csail.mit.edu/sls/downloads/movie/engtest.bio' -P './mit_ner'

# !wget 'https://raw.githubusercontent.com/masakhane-io/masakhane-ner/main/data/pcm/train.txt' -P './pcm_ner'
# !wget 'https://raw.githubusercontent.com/masakhane-io/masakhane-ner/main/data/pcm/test.txt' -P './pcm_ner'

!ls mit_ner/
# !ls pcm_ner/

--2022-08-14 18:47:34--  https://groups.csail.mit.edu/sls/downloads/movie/engtrain.bio
Resolving groups.csail.mit.edu (groups.csail.mit.edu)... 128.30.2.44
Connecting to groups.csail.mit.edu (groups.csail.mit.edu)|128.30.2.44|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1013492 (990K)
Saving to: ‘./mit_ner/engtrain.bio’

engtrain.bio        100%[===================>] 989.74K   749KB/s    in 1.3s    

2022-08-14 18:47:37 (749 KB/s) - ‘./mit_ner/engtrain.bio’ saved [1013492/1013492]

--2022-08-14 18:47:37--  https://groups.csail.mit.edu/sls/downloads/movie/engtest.bio
Resolving groups.csail.mit.edu (groups.csail.mit.edu)... 128.30.2.44
Connecting to groups.csail.mit.edu (groups.csail.mit.edu)|128.30.2.44|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252636 (247K)
Saving to: ‘./mit_ner/engtest.bio’

engtest.bio         100%[===================>] 246.71K   274KB/s    in 0.9s    

2022-08-14 18:47:39 (274 KB/s) - ‘./mit_ner/engtest.bi

Perfect. Now we see all our files. Let's poke at one of them before we start parsing our dataset.

The file is a 2-column tab separated file. The first column is the label and the second column is the token.

If you need a refresher of IOB label encoding, an O label on a token denotes that the token is not part of a named entity. A "B" label indicates that the token is the start of a named entity, with the description following the "B-" describing the type of entity. An "I" label on a token indicates that the token is part of a named entity but not the first token in the entity, with the description following the "I-" describing the type of entity.

In [ ]:
TRAIN_FILE = "mit_ner/engtrain.bio"
TEST_FILE = "mit_ner/engtest.bio"

# The file is a 2-column tab separated file. 
# The first column is the label and the second column is the token.
# Empty rows refer to a sentence break
with open(TRAIN_FILE, 'r') as file:
  reader = csv.reader(file, delimiter = '\t')
  i = 0
  while(i < 15):
    print(next(reader))
    i += 1

['O', 'what']
['O', 'movies']
['O', 'star']
['B-ACTOR', 'bruce']
['I-ACTOR', 'willis']
[]
['O', 'show']
['O', 'me']
['O', 'films']
['O', 'with']
['B-ACTOR', 'drew']
['I-ACTOR', 'barrymore']
['O', 'from']
['O', 'the']
['B-YEAR', '1980s']


Empty rows refer to a sentence break. Now we should try and parse the dataset file and create a label encoder that converts text labels to integer ids or vice versa

In [ ]:
def parse_dataset(file_path, label_encoder):
  """
  Function to parse the csv into training or test dataset

  Input: Tuple[label,token] or Empty
  Output: List of tuples of lable and token. Each list is one sentence.
  """
  data = []
  with open(file_path, 'r') as file:
    reader = csv.reader(file, delimiter = '\t')
    sentence=[]
    for row in reader:
      if len(row) == 0:
        data.append(sentence)
        sentence=[]
      else:
        sentence.append((label_encoder([row[0]])[0], row[1]))
  return data

# Custom encoder that creates the encoding in reverse order so to assign 'O' -> 0
# The default scikit LabelEncoder doesn't allow for that 
# Ref: https://stackoverflow.com/questions/51308994/python-sklearn-determine-the-encoding-order-of-labelencoder
class CustomLabelEncoder():
  def __init__(self):
    self.classes_ = None
    self.transform_map = None
    self.inverse_map = None
  
  def fit(self, y):
    self.transform_map = {}
    self.inverse_map = {}
    print(len(y))
    self.classes_ = np.unique(y)[::-1]
    for i, c in enumerate(self.classes_):
      self.transform_map[c] = i
      self.inverse_map[i] = c
    return self

  def transform(self, y):
    return [self.transform_map.get(x) for x in y]

  def inverse_transform(self, y):
    return [self.inverse_map.get(x) for x in y]

# A lable encoder converts the text labels into integer ids
def get_label_encoder():
  """Get all the labels in a dataset and return two maps that convert labels -> id or vice versa.
  """
  # We pass an identity encoder since we still need the raw labels to train the label encoder
  raw_data = parse_dataset(TRAIN_FILE, lambda x: x)
  le = CustomLabelEncoder()
  le.fit([x[0] for sentence in raw_data for x in sentence])
  return le

# Global variables used throughout the notebook
label_encoder = get_label_encoder()

99491


In [ ]:
label_encoder.classes_

array(['O', 'I-YEAR', 'I-TRAILER', 'I-TITLE', 'I-SONG', 'I-REVIEW',
       'I-RATINGS_AVERAGE', 'I-RATING', 'I-PLOT', 'I-GENRE', 'I-DIRECTOR',
       'I-CHARACTER', 'I-ACTOR', 'B-YEAR', 'B-TRAILER', 'B-TITLE',
       'B-SONG', 'B-REVIEW', 'B-RATINGS_AVERAGE', 'B-RATING', 'B-PLOT',
       'B-GENRE', 'B-DIRECTOR', 'B-CHARACTER', 'B-ACTOR'], dtype='<U17')

In [ ]:
label_encoder.transform_map

{'B-ACTOR': 24,
 'B-CHARACTER': 23,
 'B-DIRECTOR': 22,
 'B-GENRE': 21,
 'B-PLOT': 20,
 'B-RATING': 19,
 'B-RATINGS_AVERAGE': 18,
 'B-REVIEW': 17,
 'B-SONG': 16,
 'B-TITLE': 15,
 'B-TRAILER': 14,
 'B-YEAR': 13,
 'I-ACTOR': 12,
 'I-CHARACTER': 11,
 'I-DIRECTOR': 10,
 'I-GENRE': 9,
 'I-PLOT': 8,
 'I-RATING': 7,
 'I-RATINGS_AVERAGE': 6,
 'I-REVIEW': 5,
 'I-SONG': 4,
 'I-TITLE': 3,
 'I-TRAILER': 2,
 'I-YEAR': 1,
 'O': 0}

In [ ]:
label_encoder.inverse_map

{0: 'O',
 1: 'I-YEAR',
 2: 'I-TRAILER',
 3: 'I-TITLE',
 4: 'I-SONG',
 5: 'I-REVIEW',
 6: 'I-RATINGS_AVERAGE',
 7: 'I-RATING',
 8: 'I-PLOT',
 9: 'I-GENRE',
 10: 'I-DIRECTOR',
 11: 'I-CHARACTER',
 12: 'I-ACTOR',
 13: 'B-YEAR',
 14: 'B-TRAILER',
 15: 'B-TITLE',
 16: 'B-SONG',
 17: 'B-REVIEW',
 18: 'B-RATINGS_AVERAGE',
 19: 'B-RATING',
 20: 'B-PLOT',
 21: 'B-GENRE',
 22: 'B-DIRECTOR',
 23: 'B-CHARACTER',
 24: 'B-ACTOR'}

Creating the global training, validation, and test datasets from the data files.

In [ ]:
training_data = parse_dataset(TRAIN_FILE, label_encoder.transform)
test_data = parse_dataset(TEST_FILE, label_encoder.transform)

print('# of training examples: {rows}'.format(rows=len(training_data)))
print('# of test examples: {rows}'.format(rows=len(test_data)))

# of training examples: 9775
# of test examples: 2443


In [ ]:
training_data[:5]

[[(0, 'what'), (0, 'movies'), (0, 'star'), (24, 'bruce'), (12, 'willis')],
 [(0, 'show'),
  (0, 'me'),
  (0, 'films'),
  (0, 'with'),
  (24, 'drew'),
  (12, 'barrymore'),
  (0, 'from'),
  (0, 'the'),
  (13, '1980s')],
 [(0, 'what'),
  (0, 'movies'),
  (0, 'starred'),
  (0, 'both'),
  (24, 'al'),
  (12, 'pacino'),
  (0, 'and'),
  (24, 'robert'),
  (12, 'deniro')],
 [(0, 'find'),
  (0, 'me'),
  (0, 'all'),
  (0, 'of'),
  (0, 'the'),
  (0, 'movies'),
  (0, 'that'),
  (0, 'starred'),
  (24, 'harold'),
  (12, 'ramis'),
  (0, 'and'),
  (24, 'bill'),
  (12, 'murray')],
 [(0, 'find'),
  (0, 'me'),
  (0, 'a'),
  (0, 'movie'),
  (0, 'with'),
  (0, 'a'),
  (0, 'quote'),
  (0, 'about'),
  (0, 'baseball'),
  (0, 'in'),
  (0, 'it')]]

[Dataset and Data loaders](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html): Code for processing data samples can get messy and hard to maintain; we ideally want our dataset code to be decoupled from our model training code for better readability and modularity. PyTorch provides two data primitives: torch.utils.data.DataLoader and torch.utils.data.Dataset that allow you to use pre-loaded datasets as well as your own data. Dataset stores the samples and their corresponding labels, and DataLoader wraps an iterable around the Dataset to enable easy access to the samples.

[LightingDataModule](https://pytorch-lightning.readthedocs.io/en/latest/extensions/datamodules.html#datamodules): A datamodule is a shareable, reusable class that encapsulates all the steps needed to process data. A datamodule encapsulates the five steps involved in data processing in PyTorch:

1. Download / tokenize / process.
2. Clean and (maybe) save to disk.
3. Load inside Dataset.
4. Apply transforms (rotate, tokenize, etc…).
5. Wrap inside a DataLoader.


In [ ]:
from torch.utils.data import Dataset
x=Dataset(training_data[:32])

TypeError: ignored

In [ ]:
class NERDataset(Dataset):
  """Creates an pytorch dataset to consume our pre-loaded csv data

  Reference: https://pytorch.org/tutorials/beginner/basics/data_tutorial.html 
  """
  def __init__(self, data, vectorizer):
    self.dataset = data
    # Vectorizer needs to implement a vectorize function that returns vector and tokens
    # 🌟🌟🌟 Pay extra attention here since you'll have to work on this in the models 🌟🌟🌟
    self.vectorizer = vectorizer

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    words_and_label = self.dataset[idx]
    # print(words_and_label)
    # Split the input into two different lists per sentence
    labels, words = list(zip(*words_and_label))
    # print(labels,words)
    token_vectors, tokens, aligned_labels = self.vectorizer.vectorize(words, labels)
    return {
        "vectors": np.array(token_vectors),
        "tokens": tokens, # for debugging only
        "labels": aligned_labels
      }

We're going to digress for 3 seconds to learn a bit more about the sequence padding function. 

In [ ]:
a = torch.ones(22)
b = torch.ones(15)
c = torch.ones(25)
print("One dimensional Sequence", pad_sequence([a, b, c]).size())
a = torch.ones(22, 300)
b = torch.ones(25, 300)
c = torch.ones(15, 300)
print("Two dimensional Sequence", pad_sequence([a, b, c]).size())

One dimensional Sequence torch.Size([25, 3])
Two dimensional Sequence torch.Size([25, 3, 300])


In [ ]:
a = torch.ones(22, 300)
b = torch.ones(25, 300)
c = torch.ones(15, 300)
print("Two dimensional Sequence", pad_sequence([a, b, c]))

Two dimensional Sequence tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])


It'll make all the sequences of the max length in the list. Now back to writing the data module wrapper.

In [ ]:
a=torch.rand(4,2).bool()
a

tensor([[True, True],
        [True, True],
        [True, True],
        [True, True]])

In [ ]:
torch.logical_not(torch.all(a,dim=1)).int()

tensor([0, 0, 0, 0], dtype=torch.int32)

In [ ]:
class NERDataModule(pl.LightningDataModule):
  """LightningDataModule: Wrapper class for the dataset to be used in training, test, validation
  """
  def __init__(self, vectorizer, params):
    super().__init__()
    self.params = params
    self.ner_test = NERDataset(test_data, vectorizer)

    # The dataset doesn't have an explicit validation data so we split the training data into an 80-20 split
    ner_train_val_combined = NERDataset(training_data, vectorizer)

    # Compute the 80-20 split
    train_sample_size = int(len(ner_train_val_combined) * 0.8)
    validation_sample_size = len(ner_train_val_combined)- train_sample_size

    # Create the actual training and validation datasets
    self.ner_train, self.ner_val = random_split(ner_train_val_combined, [train_sample_size, validation_sample_size])
  
  # Function to convert the input raw data from the dataset into model input. 
  # 🌟🌟🌟 Pay extra attention here since you'll have to work on this in the models 🌟🌟🌟
  def collate_fn(self, batch):
    # Separate out the vectors and labels from the batch
    word_vector = [torch.Tensor(item["vectors"]) for item in batch]
    labels = [item["labels"] for item in batch]

    # Now pad each vector sequence to the same size
    padded_word_vector = pad_sequence(word_vector)

    # Max sequence length is the max size of a single sentence in this batch
    max_seq_len = padded_word_vector.shape[0]

    # Pad the label vector as well
    padded_label_vector = []
    for label in labels:
      padded_label = np.concatenate((label,np.array([0] * (max_seq_len - len(label)))))
      padded_label_vector.append(padded_label)
    padded_label_vector = torch.LongTensor(padded_label_vector)

    # Mask to make sure when computing the loss we're not accounting for the padding
    padded_mask = torch.logical_not(torch.all(torch.isclose(padded_word_vector, torch.tensor(0.0)), dim=-1)).int()

    return {"vectors": padded_word_vector, "label_ids": padded_label_vector, "mask": torch.transpose(padded_mask, 0, 1)}

  def train_dataloader(self):
    # Training dataloader .. will reset itself each epoch
    return DataLoader(self.ner_train, batch_size=self.params.batch_size, collate_fn=self.collate_fn)

  def val_dataloader(self):
    # Validation dataloader .. will reset itself each epoch
    return DataLoader(self.ner_val, batch_size=self.params.batch_size, collate_fn=self.collate_fn)

  def test_dataloader(self):
    # Test dataloader .. will reset itself each epoch
    return DataLoader(self.ner_test, batch_size=self.params.batch_size, collate_fn=self.collate_fn)

### Classfier and Trainer (Common to all solutions)

Now that we have created the DataLoader and Datasets we'll use in the entire project, it is time to write the training and testing loops. 

[LightingModule](https://pytorch-lightning.readthedocs.io/en/latest/common/lightning_module.html): organizes your PyTorch code into 5 sections

1. Computations (init).
2. Train loop (training_step)
3. Validation loop (validation_step)
4. Test loop (test_step)
5. Optimizers (configure_optimizers)

In [ ]:
np.array([0]*3)

array([0, 0, 0])

In [ ]:
# 🌟🌟🌟 Pay extra attention here since you'll have to work on this in the models 🌟🌟🌟
class NamedEntityRecognizer(pl.LightningModule):
  def __init__(self, model, params):
      super().__init__()
      self.model = model
      self.params = params
      self.validation_accuracy = torchmetrics.Accuracy(multiclass=True, mdmc_reduce="global")
      self.test_accuracy = torchmetrics.Accuracy(multiclass=True, mdmc_reduce="global")

  def forward(self, x):
      return self.model(x)

  def training_step(self, batch, batch_idx):
    x = batch["vectors"]
    y = batch["label_ids"]
    mask = batch["mask"]
    y_hat = self(x).permute(0, 2, 1)
    loss = F.cross_entropy(y_hat, y, reduction='none')
    active_loss = torch.mean(torch.multiply(loss, mask))
    self.log_dict(
        {'train_loss': active_loss}, 
        batch_size=self.params.batch_size, 
        prog_bar=True
        )
    return active_loss
  
  def validation_step(self, batch, batch_nb):
    x = batch["vectors"]
    y = batch["label_ids"]
    mask = batch["mask"]
    y_hat = self(x)
    val_loss = F.cross_entropy(y_hat.permute(0, 2, 1), y, reduction='none')
    active_loss = torch.mean(torch.multiply(val_loss, mask))
    predictions = torch.argmax(y_hat, dim=2)

    # We apply with the mask here to zero out any predictons that are just made 
    # on the padding from being added to the accuracy calculation
    self.validation_accuracy(predictions*mask, y)
    self.log_dict(
        {
          'val_loss': active_loss,
          'val_accuracy': self.validation_accuracy,
        },
        batch_size=self.params.batch_size, 
        prog_bar=True
      )
    return active_loss

  def test_step(self, batch, batch_nb):
    x = batch["vectors"]
    y = batch["label_ids"]
    mask = batch["mask"]
    y_hat = self(x)
    test_loss = F.cross_entropy(y_hat.permute(0, 2, 1), y, reduction='none')
    active_loss = torch.mean(torch.multiply(test_loss, mask))
    predictions = torch.argmax(y_hat, dim=2)
    # We apply with the mask here to zero out any predictons that are just made 
    # on the padding from being added to the accuracy calculation
    self.test_accuracy(predictions*mask, y)
    self.log_dict(
        {
          'test_loss': active_loss,
          'test_accuracy': self.test_accuracy
        }, 
        batch_size=self.params.batch_size,
        prog_bar=True
      )
    return active_loss
  
  def predict_step(self, batch, batch_idx):
    y_hat = self.model(batch["vectors"])
    predictions = torch.argmax(y_hat, dim=2)
    return {'logits':y_hat, 'predictions': predictions, 'label_ids': batch["label_ids"], 'mask': batch["mask"]}

  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr=self.params.learning_rate)
    return optimizer

We'll need a Vectorizer to convert words into word vectors. This week we'll just freeze a simple Spacy-based vectorizer and use that for all the models.

In [ ]:
class SpacyVectorizer:
  def vectorize(self, words, labels):
    """
    Given a sentence, tokenize it and returns a pre-trained word vector for each token.
    NOTE: note that the vectorizer needs to be compatible with the tokenizer.
    """
    assert len(words) == len(labels)
    sentence_vector = []
    sentence_labels = []
    sentence_tokens = []
    for i, word in enumerate(words):
      label = labels[i]

      # Tokenize the words using spacy
      spacy_doc = loaded_spacy_model.make_doc(word)
      word_vector = [token.vector for token in spacy_doc]
      word_tokens = [token.text for token in spacy_doc]
      sentence_vector += word_vector
      sentence_tokens += word_tokens

      # Make sure the length of the labels is same as length of vectors
      # In out current dataset we get pre tokenized words so this shouldn't actually happen
      word_labels = [label] * len(word_vector)
      sentence_labels += word_labels
    return sentence_vector, sentence_tokens, sentence_labels

vectorizer = SpacyVectorizer()

Once we have a Lightning and LightingDataModule, a [Trainer](https://pytorch-lightning.readthedocs.io/en/latest/common/trainer.html) automates everything else. It provides functions for training (fit), testing, and inference. We ended up writing a helper function that takes the model, vectorizer, and hyper parameters to be able to easily compare our different models.

In [ ]:
def trainer(model, params, skip_training=False):
  # Create a pytorch trainer
  trainer = pl.Trainer(max_epochs=params.max_epochs, check_val_every_n_epoch=1)

  # Initialize our data loader with the passed vectorizer
  data_module = NERDataModule(vectorizer, params)

  # Train and validate the model
  if not skip_training:
    trainer.fit(model,
                data_module.train_dataloader(), 
                val_dataloaders=data_module.val_dataloader())

  # Test the model
  trainer.test(model, data_module.test_dataloader())

  # Predict on the same test set to show some output
  output = trainer.predict(model, data_module.test_dataloader())
  
  

  for i in range(2):
    print("-----------")
    print("Label Ids: ", [label_encoder.inverse_transform([int(x)])[0] for x in output[0]['label_ids'][i]])
    print("Predictions: ", [label_encoder.inverse_transform([int(x)])[0] for x in output[0]['predictions'][i]])
    print("Mask: ", output[0]['mask'][i])

# Models

You may be wondering, "Are we building models yet?" And, the answer is YES! Finally the time has come to build our baseline model, and then we'll work towards improving it. 

### Assignment Part 1: Model 0: Make them all ZERO  -- TO BE COMPLETED
##### <font color='red'>Expected accuracy: ~78%</font>

Build a model that doesn't do any predictions and just returns 0 ('O') for all label ids to be predicted.

This is a good baseline for our model since most of our predictions are 0 anyway so we should get some decent accuracy with just this. Building a model only makes sense if we can at least do better than this :) 

In [ ]:
class HParamsZERO:
  batch_size: int = 32
  learning_rate: float = 0.02
  max_epochs: int = 4
  num_tags: int = len(label_encoder.classes_)

class NERModelZero(torch.nn.Module):
  def __init__(self, num_tags):
    super().__init__()
    self.num_tags = num_tags
    
  def forward(self, batch):
    """Batch is of shape (max_seq_len, batch_size, word_vector_dim)
    Returns: Output probabities of shape (batch_size, max_seq_len, num_tags)
    """
    ### TO BE IMPLEMENTED ###
    return torch.zeros(batch.shape[1],batch.shape[0],self.num_tags)
    ### TO BE IMPLEMENTED ###

# Train, Validate and Test the model
trainer(
    model=NamedEntityRecognizer(NERModelZero(HParamsZERO.num_tags), 
                                HParamsZERO),
    params=HParamsZERO,
    skip_training=True)  # NOTE: We skip training here since there is no variable to learn in this model

INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: False
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1585: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  "GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`."


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.7869786620140076, 'test_loss': 1.768639087677002}
--------------------------------------------------------------------------------


Predicting: 0it [00:00, ?it/s]

-----------
Label Ids:  ['O', 'O', 'O', 'O', 'B-GENRE', 'I-GENRE', 'O', 'B-YEAR', 'I-YEAR', 'O', 'O', 'O', 'O']
Predictions:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Mask:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0], dtype=torch.int32)
-----------
Label Ids:  ['O', 'O', 'O', 'O', 'O', 'B-PLOT', 'I-PLOT', 'I-PLOT', 'O', 'O', 'O', 'O', 'O']
Predictions:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Mask:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], dtype=torch.int32)


### Assignment Part 2: Model 1: Recurrent Neural Networks  -- TO BE COMPLETED
##### <font color='red'>Expected accuracy: ~91%</font>

Let's build our first [Simple RNN](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html) based model. 

Here we've two key pieces:

1. *HParams*: a class that contains all the hyper parameters we reference in the notebook.
1. *NERModelRNN*: Simple RNN model and it's forward pass implementation. The model should contain one RNN layer, followed by one dropout layer and then one linear layer to compute the output.

It will take several minutes to train the model, so don't be alarmed if you don't get the result right away. When the cell finishes running, under the `DATALOADER:0 TEST RESULTS` section.

In [ ]:
class HParamsRNN:
  batch_size: int = 32
  learning_rate: float = 0.02
  max_epochs: int = 4
  num_tags: int = len(label_encoder.classes_)
  hidden_dim: int = 128 # Added
  dropout: float = 0.2 # Added
  word_vec_dim: int = 300 # Added

class NERModelRNN(torch.nn.Module):
  def __init__(self, num_tags):
    super().__init__()
    # 🌟🌟🌟 WOOHOO!! We're using RNNs in the model
    ### TO BE IMPLEMENTED ###
    self.rnn = torch.nn.RNN(input_size=HParamsRNN.word_vec_dim,hidden_size=HParamsRNN.hidden_dim,num_layers=1,bidirectional=False)
    ### TO BE IMPLEMENTED ###
    self.dropout = torch.nn.Dropout(HParamsRNN.dropout)
    self.tags = torch.nn.Linear(HParamsRNN.hidden_dim, num_tags)
    
  def forward(self, batch):
    """Batch is of shape (max_seq_len, batch_size, word_vector_dim)
    
    Observe that each batch contains sentence encodings of equal length (the length of the longest sentence).
    This is because sentences are variable in length; however, processing sentences
    in batches via (variants of) RNNs needs to be of equal length. We have already added padding, 
    to make each batch the length of the longest sentence in the batch (see NERDataModule.collate_fn()).

    Given the batch, do the following:
    - Implement a hidden state for an RNN.
    - Use the hidden state with the batch for the RNN.
    - Put the RNN through the other layers (dropout, tags).
    - Put the outcome in the appropriate format.

    Returns: Output probabities of shape (batch_size, max_seq_len, num_tags)
    """
    # Propagate input through RNN layer
    ### TO BE IMPLEMENTED ###
    # Propagate input through RNN layer
    rnn_output,hidden_=self.rnn(batch)
    ### TO BE IMPLEMENTED ###
    
    # Propogate the rnn output through dropout and linear layers
    final_output = self.tags(self.dropout(rnn_output))
    return torch.transpose(final_output, 0, 1)

# Train, Validate and Test the model
trainer(
    model=NamedEntityRecognizer(NERModelRNN(HParamsRNN.num_tags), 
                                HParamsRNN),
    params=HParamsRNN)

INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: False
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1585: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  "GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`."
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                | Type        | Params
----------------------------------------------------
0 | model               | NERModelRNN | 58.3 K
1 | validation_accuracy | Accuracy    | 0     
2 | test_accuracy       | Accuracy    | 0     
----------------------------------------------------
58.3 K    Trainable params
0         Non-trainable params
58.3 K    Total params
0.233     Total estimated mod

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.906148374080658, 'test_loss': 0.39476272463798523}
--------------------------------------------------------------------------------


Predicting: 245it [00:00, ?it/s]

-----------
Label Ids:  ['O', 'O', 'O', 'O', 'B-GENRE', 'I-GENRE', 'O', 'B-YEAR', 'I-YEAR', 'O', 'O', 'O', 'O']
Predictions:  ['O', 'O', 'O', 'B-RATINGS_AVERAGE', 'B-GENRE', 'I-GENRE', 'O', 'O', 'O', 'O', 'O', 'O', 'I-TITLE']
Mask:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0], dtype=torch.int32)
-----------
Label Ids:  ['O', 'O', 'O', 'O', 'O', 'B-PLOT', 'I-PLOT', 'I-PLOT', 'O', 'O', 'O', 'O', 'O']
Predictions:  ['O', 'O', 'O', 'O', 'O', 'B-PLOT', 'O', 'O', 'O', 'I-TITLE', 'I-TITLE', 'I-TITLE', 'B-YEAR']
Mask:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], dtype=torch.int32)


🎉🎉🎉 WE HAVE OUR NER Model that uses RNNs 🎉🎉🎉

Now might be a good time to 1) celebrate!, and 2) re-read the code to internalize it.

### Assignment Part 3: Model 2: Bidirectional Recurrent Neural Networks -- TO BE COMPLETED
##### <font color='red'>Expected accuracy: ~90%</font>

Now that you've got a [simple RNN](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html) based model working, we like to add one small addition. This addition, as you might've read from the PyTorch documentation, you can apply to any RNN based model. 

We do this by adding a few parameters which are used to implement bidirectionality for a RNN model.

In [ ]:
from torch.nn.modules import dropout
class HParamsBiRNN:
  batch_size: int = 32
  learning_rate: float = 0.02
  max_epochs: int = 4
  num_tags: int = len(label_encoder.classes_)
  hidden_dim: int = 128
  dropout: float = 0.2
  word_vec_dim: int = 300
  num_layers: int = 2 # Added
  bidirectional: bool = True # Added

class NERModelRNN(torch.nn.Module):
  def __init__(self, num_tags):
    super().__init__()
    ### TO BE IMPLEMENTED ###
    self.brnn=torch.nn.RNN(input_size=HParamsBiRNN.word_vec_dim,hidden_size=HParamsBiRNN.hidden_dim,num_layers=2,bidirectional=True,dropout=HParamsBiRNN.dropout,batch_first=True)
    ### TO BE IMPLEMENTED ###
    self.dropout = torch.nn.Dropout(HParamsBiRNN.dropout)
    self.tags = torch.nn.Linear(2*HParamsBiRNN.hidden_dim, num_tags)

    
  def forward(self, batch):
    """Batch is of shape (max_seq_len, batch_size, word_vector_dim)

    Returns: Output probabities of shape (batch_size, max_seq_len, num_tags)
    """
    # Propagate input through RNN layer
    max_seq_len = batch.shape[0]
    ### TO BE IMPLEMENTED ###
    brnn_outputs,hidden_=self.brnn(batch)
    ### TO BE IMPLEMENTED ###
    # Propogate the rnn output through dropout and linear layers
    final_output = self.tags(self.dropout(brnn_outputs))
    return torch.transpose(final_output, 0, 1)

# Train, Validate and Test the model
trainer(
    model=NamedEntityRecognizer(NERModelRNN(HParamsBiRNN.num_tags), 
                                HParamsBiRNN),
    params=HParamsBiRNN)

INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: False
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1585: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  "GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`."
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                | Type        | Params
----------------------------------------------------
0 | model               | NERModelRNN | 215 K 
1 | validation_accuracy | Accuracy    | 0     
2 | test_accuracy       | Accuracy    | 0     
----------------------------------------------------
215 K     Trainable params
0         Non-trainable params
215 K     Total params
0.861     Total estimated mod

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.8427221179008484, 'test_loss': 0.6801526546478271}
--------------------------------------------------------------------------------


Predicting: 245it [00:00, ?it/s]

-----------
Label Ids:  ['O', 'O', 'O', 'O', 'B-GENRE', 'I-GENRE', 'O', 'B-YEAR', 'I-YEAR', 'O', 'O', 'O', 'O']
Predictions:  ['O', 'O', 'O', 'O', 'B-GENRE', 'B-GENRE', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Mask:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0], dtype=torch.int32)
-----------
Label Ids:  ['O', 'O', 'O', 'O', 'O', 'B-PLOT', 'I-PLOT', 'I-PLOT', 'O', 'O', 'O', 'O', 'O']
Predictions:  ['O', 'O', 'O', 'O', 'O', 'B-PLOT', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Mask:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], dtype=torch.int32)


Nice! Do we see any performance improvements? What will the change do to the runtime? 

We'd like to encourage you to add this option to later models as well and compare.

### Assignment Part 4: Model 3: LSTMs ---- TO BE COMPLETED
##### <font color='red'>Expected accuracy: ~95%</font>

We'll be re-using the simple RNN model from Model-1 but changing the layers from RNN to LSTMs. 

1. Change the RNN layer to [LSTM](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html#torch.nn.LSTM) Layer
2. Keep the dropout and linear layers in the model
3. Complete the forward pass implementation

Does this model perform better than our baseline?

In [ ]:
class HParamsLSTM:
  batch_size: int = 32
  learning_rate: float = 0.02
  max_epochs: int = 4
  num_tags: int = len(label_encoder.classes_)
  hidden_dim: int = 128
  dropout: float = 0.2
  word_vec_dim: int = 300
  num_layers: int = 1 # Added
  ### TO BE IMPLEMENTED --- Some parameters are missing here so add them ##
 
  ### TO BE IMPLEMENTED ###

class NERModelLSTM(torch.nn.Module):
  def __init__(self, num_tags):
    """
    Three layers: LSTM, dropout and linear chained one after the other.
    """
    super().__init__()
    ### TO BE IMPLEMENTED ###
    self.hidden_dim = HParamsLSTM.hidden_dim
    self.layer_dim = HParamsLSTM.num_layers
    ### TO BE IMPLEMENTED ###

    self.lstm = lstm =torch.nn.LSTM(HParamsLSTM.word_vec_dim, HParamsLSTM.hidden_dim, HParamsLSTM.num_layers)
    self.dropout = torch.nn.Dropout(HParamsLSTM.dropout)
    self.tags = torch.nn.Linear(HParamsLSTM.hidden_dim, HParamsLSTM.num_tags)

  def forward(self, batch):
    """Batch is of shape (max_seq_len, batch_size, word_vector_dim)
    Returns: Output probabities of shape (batch_size, max_seq_len, num_tags)
    """
    # Propagate input through LSTM

    ### TO BE IMPLEMENTED ###
    lstm_outputs,hidden_= self.lstm(batch)
    ### TO BE IMPLEMENTED ###

    
    ### TO BE IMPLEMENTED ###
    final_output = self.tags(self.dropout(lstm_outputs))
    ### TO BE IMPLEMENTED ###
  
    
    return torch.transpose(final_output, 0, 1)


# Train, Validate and Test the model
trainer(
    model=NamedEntityRecognizer(NERModelLSTM(len(label_encoder.classes_)), 
                                HParamsLSTM),
    params=HParamsLSTM)

INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: False
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1585: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  "GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`."
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                | Type         | Params
-----------------------------------------------------
0 | model               | NERModelLSTM | 223 K 
1 | validation_accuracy | Accuracy     | 0     
2 | test_accuracy       | Accuracy     | 0     
-----------------------------------------------------
223 K     Trainable params
0         Non-trainable params
223 K     Total params
0.894     Total estimat

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.9451513886451721, 'test_loss': 0.2230260968208313}
--------------------------------------------------------------------------------


Predicting: 245it [00:00, ?it/s]

-----------
Label Ids:  ['O', 'O', 'O', 'O', 'B-GENRE', 'I-GENRE', 'O', 'B-YEAR', 'I-YEAR', 'O', 'O', 'O', 'O']
Predictions:  ['O', 'O', 'O', 'O', 'B-GENRE', 'I-GENRE', 'O', 'O', 'O', 'B-PLOT', 'I-PLOT', 'I-PLOT', 'I-PLOT']
Mask:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0], dtype=torch.int32)
-----------
Label Ids:  ['O', 'O', 'O', 'O', 'O', 'B-PLOT', 'I-PLOT', 'I-PLOT', 'O', 'O', 'O', 'O', 'O']
Predictions:  ['O', 'O', 'O', 'O', 'O', 'B-PLOT', 'O', 'O', 'B-PLOT', 'I-PLOT', 'I-PLOT', 'I-PLOT', 'I-PLOT']
Mask:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], dtype=torch.int32)


Play around with some predictions? On what examples does the LSTM do much better than the RNN?

### Assignment Part 5: Model 4: Stacked LSTMs ---- TO BE COMPLETED
##### <font color='red'>Both expected accuracy: ~95%</font>

So far we have created single layer LSTM and RNNs. Now change the code to create a new stacked LSTM with 2 and 4. How do they perform? 


In [ ]:
# 2 Stacked LSTM
class HParamsLSTMS2:
  batch_size: int = 32
  learning_rate: float = 0.02
  max_epochs: int = 4
  max_seq_len: int = 20
  hidden_dim: int = 128
  word_vec_dim: int = 300
  num_layers: int = 2 # Adapt
  dropout: float = 0.2
  num_tags: int = len(label_encoder.classes_)

# Train, Validate and Test the model
trainer(
    model=NamedEntityRecognizer(NERModelLSTM(HParamsLSTMS2.num_tags), 
                                HParamsLSTMS2),
    params=HParamsLSTMS2)

INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: False
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1585: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  "GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`."
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                | Type         | Params
-----------------------------------------------------
0 | model               | NERModelLSTM | 223 K 
1 | validation_accuracy | Accuracy     | 0     
2 | test_accuracy       | Accuracy     | 0     
-----------------------------------------------------
223 K     Trainable params
0         Non-trainable params
223 K     Total params
0.894     Total estimat

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.9469628930091858, 'test_loss': 0.21803340315818787}
--------------------------------------------------------------------------------


Predicting: 245it [00:00, ?it/s]

-----------
Label Ids:  ['O', 'O', 'O', 'O', 'B-GENRE', 'I-GENRE', 'O', 'B-YEAR', 'I-YEAR', 'O', 'O', 'O', 'O']
Predictions:  ['O', 'O', 'O', 'O', 'B-GENRE', 'I-GENRE', 'O', 'O', 'O', 'B-PLOT', 'I-PLOT', 'I-PLOT', 'I-PLOT']
Mask:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0], dtype=torch.int32)
-----------
Label Ids:  ['O', 'O', 'O', 'O', 'O', 'B-PLOT', 'I-PLOT', 'I-PLOT', 'O', 'O', 'O', 'O', 'O']
Predictions:  ['O', 'O', 'O', 'O', 'O', 'B-PLOT', 'I-PLOT', 'I-PLOT', 'I-PLOT', 'I-PLOT', 'I-PLOT', 'I-PLOT', 'I-PLOT']
Mask:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], dtype=torch.int32)


In [ ]:
# 4 Stacked LSTM
class HParamsLSTMS4:
  batch_size: int = 32
  learning_rate: float = 0.02
  max_epochs: int = 4
  max_seq_len: int = 20
  hidden_dim: int = 128
  word_vec_dim: int = 300
  num_layers: int = 4 # Adapt
  dropout: float = 0.2
  num_tags: int = len(label_encoder.classes_)

# Train, Validate and Test the model
trainer(
    model=NamedEntityRecognizer(NERModelLSTM(HParamsLSTMS4.num_tags), 
                                HParamsLSTMS4),
    params=HParamsLSTMS4)

INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: False
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1585: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  "GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`."
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                | Type         | Params
-----------------------------------------------------
0 | model               | NERModelLSTM | 223 K 
1 | validation_accuracy | Accuracy     | 0     
2 | test_accuracy       | Accuracy     | 0     
-----------------------------------------------------
223 K     Trainable params
0         Non-trainable params
223 K     Total params
0.894     Total estimat

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.9489490985870361, 'test_loss': 0.2142360806465149}
--------------------------------------------------------------------------------


Predicting: 245it [00:00, ?it/s]

-----------
Label Ids:  ['O', 'O', 'O', 'O', 'B-GENRE', 'I-GENRE', 'O', 'B-YEAR', 'I-YEAR', 'O', 'O', 'O', 'O']
Predictions:  ['O', 'O', 'O', 'O', 'B-GENRE', 'I-GENRE', 'O', 'O', 'O', 'B-TITLE', 'I-PLOT', 'I-PLOT', 'I-PLOT']
Mask:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0], dtype=torch.int32)
-----------
Label Ids:  ['O', 'O', 'O', 'O', 'O', 'B-PLOT', 'I-PLOT', 'I-PLOT', 'O', 'O', 'O', 'O', 'O']
Predictions:  ['O', 'O', 'O', 'O', 'O', 'B-PLOT', 'O', 'O', 'I-PLOT', 'I-PLOT', 'I-PLOT', 'I-PLOT', 'I-PLOT']
Mask:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], dtype=torch.int32)


### Assignment Part 6: Is Accuracy a good metric? 

We've been using Accuracy as a metric for the last two weeks. Is this a good metric for this problem? Debate and write a short answer below:

---

**YOUR ANSWER TO BE FILLED HERE**

---
Accuracy is not a good metric for datasets with highly imbalanced examples.This is because even a bad model would have a high accuracy because the dataset is skewed to one side/class. SO F1 score is a better metric than accuracy in this kind of prblem.


### Assignment Part 7: Implement a new LightningModule. --- TO BE COMPLETED
##### <font color='red'>Expected accuracy: ~79%, expected F1: ~4%</font>

We're going to compute more metrics on our models. Let's create a new training module which adds **F1 Score** to it.

**NamedEntityRecognizerWithF1**: Implement some functions we've seen multiple times now

1. Initialize the F1 metric correctly with *mdmc_average:global* and *average:macro*.
1. `validation_step`: Write the function that computes validation_loss. It should be really close to what you've seen before.
1. `test_step`: Write the function that computes test_loss. It should be really close to what you've seen before.
1. `predict_step`: Implement a function that does inference here. Will anything need to change here?
1. `configure_optimizers`: Configure adam optimizer to run at the learning rate from the hyper parameters. Will anything need to change here?


Good explanation: [of micro vs macro averages for multiclass classification](https://datascience.stackexchange.com/questions/15989/micro-average-vs-macro-average-performance-in-a-multiclass-classification-settin/24051#24051)

Another reading for metrics on multi-class classification is [here](https://towardsdatascience.com/comprehensive-guide-on-multiclass-classification-metrics-af94cfb83fbd)


In [ ]:
from torchmetrics import F1Score


class HParamsF1:
  batch_size: int = 32
  learning_rate: float = 0.02
  max_epochs: int = 4
  max_seq_len: int = 20
  hidden_dim: int = 128
  word_vec_dim: int = 300
  num_layers: int = 1
  dropout: float = 0.2
  num_tags: int = len(label_encoder.classes_)

class NamedEntityRecognizerWithF1(pl.LightningModule):
  def __init__(self, model, params):
      super().__init__()
      self.model = model
      self.params = params
      self.validation_accuracy = torchmetrics.Accuracy(multiclass=True, mdmc_reduce="global")
      self.test_accuracy = torchmetrics.Accuracy(multiclass=True, mdmc_reduce="global")
      ### ADD YOUR NEW METRICS ### 
      ### TO BE IMPLEMENTED ###
      self.val_f1score = torchmetrics.F1Score(num_classes=len(label_encoder.classes_), mdmc_reduce="global", average="macro")
      self.test_f1score = torchmetrics.F1Score(num_classes=len(label_encoder.classes_), mdmc_reduce="global", average="macro")
      ### TO BE IMPLEMENTED ###  

  def forward(self, x):
      return self.model(x)

  def training_step(self, batch, batch_idx):
    ### TO BE IMPLEMENTED ###
    x = batch["vectors"]
    y = batch["label_ids"]
    mask = batch["mask"]
    y_hat = self(x).permute(0, 2, 1)
    loss = F.cross_entropy(y_hat, y, reduction='none')
    active_loss = torch.mean(torch.multiply(loss, mask))
    self.log_dict(
        {'train_loss': active_loss}, 
        batch_size=self.params.batch_size, 
        prog_bar=True
        )
    ### TO BE IMPLEMENTED ###
    return active_loss
  
  def validation_step(self, batch, batch_nb):
    ### TO BE IMPLEMENTED ###
    x = batch["vectors"]
    y = batch["label_ids"]
    mask = batch["mask"]
    y_hat = self(x)
    val_loss = F.cross_entropy(y_hat.permute(0, 2, 1), y, reduction='none')
    active_loss = torch.mean(torch.multiply(val_loss, mask))
    predictions = torch.argmax(y_hat, dim=2)

    # We apply with the mask here to zero out any predictons that are just made 
    # on the padding from being added to the accuracy calculation
    self.validation_accuracy(predictions*mask, y)
    # pred = y_hat.softmax(dim=-1)
    self.val_f1score(predictions*mask, y)
    self.log_dict(
        {
          'val_loss': active_loss,
          'val_accuracy': self.validation_accuracy,
          'val_f1_score': self.val_f1score,
        },
        batch_size=self.params.batch_size, 
        prog_bar=True
      )
    ### TO BE IMPLEMENTED ###
    return active_loss

  def test_step(self, batch, batch_nb):
    ### TO BE IMPLEMENTED ###
    x = batch["vectors"]
    y = batch["label_ids"]
    mask = batch["mask"]
    y_hat = self(x)
    test_loss = F.cross_entropy(y_hat.permute(0, 2, 1), y, reduction='none')
    active_loss = torch.mean(torch.multiply(test_loss, mask))
    predictions = torch.argmax(y_hat, dim=2)
    # We apply with the mask here to zero out any predictons that are just made 
    # on the padding from being added to the accuracy calculation
    self.test_accuracy(predictions*mask, y)
    pred = y_hat.softmax(dim=-1)
    pred= torch.argmax(pred, dim=2)
    # print(pred.shape, y_hat.shape, y.shape, predictions.shape)
    self.test_f1score(pred*mask, y)
    self.log_dict(
        {
          'test_loss': active_loss,
          'test_accuracy': self.test_accuracy,
          'test_f1_score': self.test_f1score,
        }, 
        batch_size=self.params.batch_size,
        prog_bar=True
      )
    ### TO BE IMPLEMENTED ###
    return active_loss
  
  def predict_step(self, batch, batch_idx):
    ### TO BE IMPLEMENTED ###
    y_hat = self.model(batch["vectors"])
    predictions = torch.argmax(y_hat, dim=2)
    return {'logits':y_hat, 'predictions': predictions, 'label_ids': batch["label_ids"], 'mask': batch["mask"]}
    ### TO BE IMPLEMENTED ###

  def configure_optimizers(self):
    ### TO BE IMPLEMENTED ###
    optimizer = torch.optim.Adam(self.parameters(), lr=self.params.learning_rate)
    return optimizer
    ### TO BE IMPLEMENTED ###

# Train, Validate and Test the model
trainer(
    model=NamedEntityRecognizerWithF1(NERModelZero(HParamsF1.num_tags), HParamsF1),
    params=HParamsF1,
    skip_training=True)  # NOTE: We skip training here since there is no variable to learn in this model

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1585: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  "GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`."


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.7869786620140076,
 'test_f1_score': 0.03523169830441475,
 'test_loss': 1.768639087677002}
--------------------------------------------------------------------------------


Predicting: 0it [00:00, ?it/s]

-----------
Label Ids:  ['O', 'O', 'O', 'O', 'B-GENRE', 'I-GENRE', 'O', 'B-YEAR', 'I-YEAR', 'O', 'O', 'O', 'O']
Predictions:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Mask:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0], dtype=torch.int32)
-----------
Label Ids:  ['O', 'O', 'O', 'O', 'O', 'B-PLOT', 'I-PLOT', 'I-PLOT', 'O', 'O', 'O', 'O', 'O']
Predictions:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Mask:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], dtype=torch.int32)


#### Let's re-run our RNN and LSTM models once with the new metrics
##### <font color='red'>Expected accuracy: ~88%, expected F1: ~35%</font>
##### <font color='red'>Expected accuracy: ~95%, expected F1: ~69%</font>

In [ ]:
trainer(
    model=NamedEntityRecognizerWithF1(NERModelRNN(HParamsRNN.num_tags), HParamsRNN),
    params=HParamsRNN)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1585: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  "GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`."

  | Name                | Type        | Params
----------------------------------------------------
0 | model               | NERModelRNN | 215 K 
1 | validation_accuracy | Accuracy    | 0     
2 | test_accuracy       | Accuracy    | 0     
3 | val_f1score         | F1Score     | 0     
4 | test_f1score        | F1Score     | 0     
----------------------------------------------------
215 K     Trainable params
0         Non-trainable params
215 K     Total params
0.861     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.8497501015663147,
 'test_f1_score': 0.27800893783569336,
 'test_loss': 0.6376940011978149}
--------------------------------------------------------------------------------


Predicting: 245it [00:00, ?it/s]

-----------
Label Ids:  ['O', 'O', 'O', 'O', 'B-GENRE', 'I-GENRE', 'O', 'B-YEAR', 'I-YEAR', 'O', 'O', 'O', 'O']
Predictions:  ['O', 'O', 'O', 'O', 'B-GENRE', 'B-GENRE', 'O', 'O', 'O', 'B-PLOT', 'B-PLOT', 'B-PLOT', 'B-PLOT']
Mask:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0], dtype=torch.int32)
-----------
Label Ids:  ['O', 'O', 'O', 'O', 'O', 'B-PLOT', 'I-PLOT', 'I-PLOT', 'O', 'O', 'O', 'O', 'O']
Predictions:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-TITLE', 'I-TITLE', 'I-TITLE', 'I-TITLE']
Mask:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], dtype=torch.int32)


In [ ]:
trainer(
    model=NamedEntityRecognizerWithF1(NERModelLSTM(HParamsLSTM.num_tags), HParamsLSTM),
    params=HParamsLSTM)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1585: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  "GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`."

  | Name                | Type         | Params
-----------------------------------------------------
0 | model               | NERModelLSTM | 223 K 
1 | validation_accuracy | Accuracy     | 0     
2 | test_accuracy       | Accuracy     | 0     
3 | val_f1score         | F1Score      | 0     
4 | test_f1score        | F1Score      | 0     
-----------------------------------------------------
223 K     Trainable params
0         Non-trainable params
223 K     Total params
0.894     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.9433180093765259,
 'test_f1_score': 0.6711282730102539,
 'test_loss': 0.2357475906610489}
--------------------------------------------------------------------------------


Predicting: 245it [00:00, ?it/s]

-----------
Label Ids:  ['O', 'O', 'O', 'O', 'B-GENRE', 'I-GENRE', 'O', 'B-YEAR', 'I-YEAR', 'O', 'O', 'O', 'O']
Predictions:  ['O', 'O', 'O', 'O', 'B-GENRE', 'I-GENRE', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Mask:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0], dtype=torch.int32)
-----------
Label Ids:  ['O', 'O', 'O', 'O', 'O', 'B-PLOT', 'I-PLOT', 'I-PLOT', 'O', 'O', 'O', 'O', 'O']
Predictions:  ['O', 'O', 'O', 'O', 'O', 'B-PLOT', 'O', 'O', 'I-PLOT', 'I-PLOT', 'I-PLOT', 'I-PLOT', 'I-PLOT']
Mask:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], dtype=torch.int32)


🎉 CONGRATS on finishing the assignment!!! Now is a good time to pause and reflect how much progress we've made in understanding NER, using RNNs, and LSTMs. But hey, don't stop here! There is a lot to do or play with in the next section.

---

# Extensions

Now that you've worked through the project. There is a lot more for us to try.

- See if you can use a LSTM to improve the model you shipped in Week 1.
- Which model performed the best? Why do you think that was?
- Why and how many stacks can we keep adding here ?
- Experiment with the hyper parameters to see how much can you improve each of the models
- Use a conditional random field (CRF) along with LSTM. [Tutorial](https://towardsdatascience.com/conditional-random-field-tutorial-in-pytorch-ca0d04499463)
- Visualize a [confusion matrix](https://torchmetrics.readthedocs.io/en/latest/references/functional.html#confusion-matrix-func) of N*N of actual class vs predicted class (N = number of classes)
- Try out other [NER datasets](https://github.com/juand-r/entity-recognition-datasets) on our models.